In [463]:
import pandas as pd

df = pd.read_parquet('./data/')
# display(df)
df.drop('0_price', axis=1, inplace=True)
df.drop('1_price', axis=1, inplace=True)
df.drop('2_price', axis=1, inplace=True)
df.drop('3_price', axis=1, inplace=True)
df.drop('4_price', axis=1, inplace=True)
#
# for col in df.columns:
#     if col[1] == '_':
#         df.drop(col, axis=1, inplace=True)
#
# df.drop('stabilized_nr_trades', axis=1, inplace=True)
# df.drop('stabilized_amount_mean', axis=1, inplace=True)
# df.drop('stabilized_at_ms', axis=1, inplace=True)
#
# df.loc[df['wave_direction'] == 'min', 'wave_direction'] = 1
# df.loc[df['wave_direction'] == 'max', 'wave_direction'] = -1
# print(f'max spread: {df["stabilized_spread"].max()}')
# df = df[df['stabilized_spread'] >= 0.2]
# df = df[df['stabilized_spread'] < 6]
# # df = df[df['last_price_delta_since_stabilized'] < 5]
# df = df.reset_index(drop=True)

df.loc[df['wave_direction'] == 'min', 'wave_direction'] = 1
df.loc[df['wave_direction'] == 'max', 'wave_direction'] = -1
df = df.astype({"wave_direction": 'float64'})

df = df[abs(df['last_price_delta_since_stabilized']) < 10]
df = df[abs(df['last_price_delta_since_stabilized']) > 0.2]
df = df[df['stabilized_spread'] < 10]  # nem tul nagy spread
df = df[df['stabilized_amount_mean'] < 0.4]  # nem tul nagy tradek
df = df[df['stabilized_nr_trades'] < 300]  # nem tul sok trade

for col in df.columns:
    if 'nr_trades' in col or 'price_delta' in col and not col.startswith('last'):
        df.drop(col, axis=1, inplace=True)
# df.drop('wave_direction', axis=1, inplace=True)

df.reset_index(drop=True, inplace=True)

display(df)

,0_amount_mean,0_spread,1_amount_mean,1_spread,2_amount_mean,2_spread,3_amount_mean,3_spread,4_amount_mean,4_spread,last_price_delta_since_stabilized,stabilized_amount_mean,stabilized_at_ms,stabilized_spread,wave_direction
0,0.0140,0.37,0.0125,0.37,0.0176,0.62,0.0491,0.67,0.0470,0.77,-0.36,0.0448,214,0.77,-1.0
1,0.0270,0.48,0.0463,0.52,0.0423,0.52,0.0462,0.72,0.0437,0.76,0.51,0.0415,91,0.79,1.0
2,0.0012,0.44,0.0022,0.57,0.0019,0.57,0.0019,0.58,0.0034,0.78,0.42,0.0046,221,0.78,1.0
3,0.0417,0.69,0.0221,0.79,0.0202,0.79,0.0573,0.79,0.0563,0.87,-0.49,0.0526,300,0.95,-1.0
4,0.0053,0.33,0.0569,0.39,0.0551,0.39,0.0527,0.49,0.0534,0.51,-0.44,0.0488,185,0.51,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14527,0.0256,0.30,0.0349,0.31,0.0275,0.45,0.0338,0.45,0.0297,0.56,-1.03,0.0283,57,0.62,-1.0
14528,0.0043,0.04,0.0042,0.04,0.0038,0.23,0.0034,0.25,0.0031,0.25,0.49,0.0038,378,0.44,1.0
14529,0.0782,0.35,0.0696,0.50,0.0664,0.56,0.0639,0.57,0.0587,0.58,1.76,0.0549,284,0.61,1.0
14530,0.0082,0.66,0.0053,0.66,0.0048,0.66,0.0044,0.66,0.0041,0.66,0.79,0.0116,273,0.83,1.0


In [464]:
df['last_price_delta_since_stabilized'].max()

9.99

In [465]:
from sklearn.model_selection import train_test_split

y = df['last_price_delta_since_stabilized']
X = df.drop('last_price_delta_since_stabilized', axis=1)
X = X.reset_index(drop=True)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
X

,0_amount_mean,0_spread,1_amount_mean,1_spread,2_amount_mean,2_spread,3_amount_mean,3_spread,4_amount_mean,4_spread,stabilized_amount_mean,stabilized_at_ms,stabilized_spread,wave_direction
0,0.0140,0.37,0.0125,0.37,0.0176,0.62,0.0491,0.67,0.0470,0.77,0.0448,214,0.77,-1.0
1,0.0270,0.48,0.0463,0.52,0.0423,0.52,0.0462,0.72,0.0437,0.76,0.0415,91,0.79,1.0
2,0.0012,0.44,0.0022,0.57,0.0019,0.57,0.0019,0.58,0.0034,0.78,0.0046,221,0.78,1.0
3,0.0417,0.69,0.0221,0.79,0.0202,0.79,0.0573,0.79,0.0563,0.87,0.0526,300,0.95,-1.0
4,0.0053,0.33,0.0569,0.39,0.0551,0.39,0.0527,0.49,0.0534,0.51,0.0488,185,0.51,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14527,0.0256,0.30,0.0349,0.31,0.0275,0.45,0.0338,0.45,0.0297,0.56,0.0283,57,0.62,-1.0
14528,0.0043,0.04,0.0042,0.04,0.0038,0.23,0.0034,0.25,0.0031,0.25,0.0038,378,0.44,1.0
14529,0.0782,0.35,0.0696,0.50,0.0664,0.56,0.0639,0.57,0.0587,0.58,0.0549,284,0.61,1.0
14530,0.0082,0.66,0.0053,0.66,0.0048,0.66,0.0044,0.66,0.0041,0.66,0.0116,273,0.83,1.0


In [466]:
from datetime import datetime
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(n_estimators=100, random_state=0, max_depth=5)
model.fit(X_train, y_train)

preds = model.predict(X_valid)
mae = round(mean_absolute_error(y_valid, preds), 5)
display(f'mean absolute error: {mae}')

display(pd.DataFrame(preds).head(8))
yy = pd.DataFrame(y_valid)
yy.reset_index(inplace=True, drop=True)
display(yy.head(8))
# y_valid.reset_index(drop=True)
#

'mean absolute error: 1.15367'

,0
0,-1.497966
1,1.129137
2,-1.870534
3,-1.394963
4,2.247875
5,1.776306
6,-1.933266
7,-1.152333


,last_price_delta_since_stabilized
0,-1.94
1,0.87
2,-1.01
3,-0.43
4,1.94
5,3.12
6,-1.41
7,-1.93


## catboost

In [468]:
from catboost import Pool

train_pool = Pool(X_train, y_train)
# train_pool = Pool(X_train, y_train, cat_features=CATEGORIES)
dev_pool = Pool(X_valid, y_valid)
# dev_pool = Pool(X_valid, y_valid, cat_features=CATEGORIES)

model.fit(train_pool, eval_set=dev_pool)
print("RMSE:", model.get_best_score())
print("result:", model.get_test_eval())


TypeError: BaseForest.fit() got an unexpected keyword argument 'eval_set'

## save model

In [ ]:

rec = len(X_train)
filename = f'models/{datetime.utcnow().replace(microsecond=0).isoformat()}-rec-{rec}-mae-{mae}.sav'
pickle.dump(model, open(filename, 'wb'))
print(f'saved model to {filename}')

In [ ]:
model = pickle.load(open(filename, 'rb'))
wave_frame = [{'0_amount_mean': 0.026, '0_nr_trades': 4.0, '0_price_delta': 1.38, '0_spread': 0.02, '1_amount_mean': 0.0169, '1_nr_trades': 7.0, '1_price_delta': 1.32, '1_spread': 0.08, '2_amount_mean': 0.016, '2_nr_trades': 8.0,
               '2_price_delta': 0.35, '2_spread': 1.05, '3_amount_mean': 0.0144, '3_nr_trades': 9.0, '3_price_delta': 0.35, '3_spread': 1.05, '4_amount_mean': 0.0123, '4_nr_trades': 11.0, '4_price_delta': 0.07, '4_spread': 1.33,
               'last_price_delta_since_stabilized': -0.02, 'stabilized_amount_mean': 0.0114, 'stabilized_at_ms': 243, 'stabilized_nr_trades': 12.0, 'stabilized_spread': 1.4, 'wave_direction': 'max'}]
df = pd.DataFrame(wave_frame)
df.loc[df['wave_direction'] == 'min', 'wave_direction'] = 1
df.loc[df['wave_direction'] == 'max'] = -1
df.drop('last_price_delta_since_stabilized', axis=1, inplace=True)
result = model.predict(df)
print(result)
